---
Setup

In [1]:
!cat /home/project/ml/pytorch/torch/version.py

__version__ = '2.0.0a0+git7f72623'
debug = False
cuda = None
git_version = '7f72623c084769ef560af35e2a0178c06492f24f'
hip = None


In [2]:
#pragma cling add_include_path("/home/project/ml/pytorch/torch/include")
#pragma cling add_include_path("/home/project/ml/pytorch/torch/include/torch/csrc/api/include")
// If you want to add library path
#pragma cling add_library_path("/home/project/ml/pytorch/torch/lib")
// If you want to load library
#pragma cling load("libtorch")
#pragma cling load("libtorch_cpu")
#pragma cling load("libc10")

----

Test

In [3]:
#include <iostream>
#include <ATen/ATen.h>

auto p = at::CPU(at::kFloat);
std::cout << p << "\n";
auto t = at::ones({3, 4}, p);
std::cout << t << "\n";

CPUFloatType
 1  1  1  1
 1  1  1  1
 1  1  1  1
[ CPUFloatType{3,4} ]


In [4]:
std::cout << t.sizes()

[3, 4]

@0x7f0c2c693b60

In [5]:
t.dim()

2

In [6]:
std::cout << t.dtype()

@0x7f0c2c693b60

In [9]:
t.is_empty()

input_line_20:2:4: error: no member named 'is_empty' in 'at::Tensor'
 t.is_empty()
 ~ ^


Interpreter Error: 

### Can we inplace replace float values by int ?

like https://github.com/uploadcare/pillow-simd/blob/668aa48d12305b8f093958792a5e4f690c2583d6/src/libImaging/Resample.c#L282-L306

```
    // use the same buffer for normalized coefficients
    INT16 *kk = (INT16 *) prekk;

    maxkk = prekk[0];
    for (x = 0; x < outSize * ksize; x++) {
        if (maxkk < prekk[x]) {
            maxkk = prekk[x];
        }
    }

    for (coefs_precision = 0; coefs_precision < PRECISION_BITS; coefs_precision += 1) {
        int next_value = (int) (0.5 + maxkk * (1 << (coefs_precision + 1)));
        // The next value will be outside of the range, so just stop
        if (next_value >= (1 << MAX_COEFS_PRECISION))
            break;
    }

    for (x = 0; x < outSize * ksize; x++) {
        if (prekk[x] < 0) {
            kk[x] = (int) (-0.5 + prekk[x] * (1 << coefs_precision));
        } else {
            kk[x] = (int) (0.5 + prekk[x] * (1 << coefs_precision));
        }
    }
    return coefs_precision;
```

In [7]:
at::manual_seed(12);
auto t = (at::rand(20) - 0.5) * 0.345;
auto tt = t.clone();

In [8]:
std::cout << t

-0.0118
-0.0922
-0.0163
 0.0301
-0.0315
-0.1286
 0.0474
-0.0890
 0.0798
 0.0767
-0.1038
 0.0672
 0.0286
 0.0455
 0.0193
-0.1289
 0.1653
 0.1188
-0.1292
-0.0188
[ CPUFloatType{20} ]

@0x7fe8ec920b60

In [9]:
float tmax = t.max().item<float>();
int weights_precision = 16;

In [10]:
tmax

0.165265f

In [11]:
weights_precision

16

In [12]:
short * data_i16 = (short*) t.data_ptr<float>();
float * data_f32 = t.data_ptr<float>();

In [13]:
data_f32[0]

-0.0118378f

In [14]:
data_i16[0]

-3213

In [15]:
for (int i=0; i<20; i++) {
    float v = data_f32[i];
    if (v < 0) {
        data_i16[i] = (int) (-0.5 + v * (1 << weights_precision));
    } else {
        data_i16[i] = (int) (0.5 + v * (1 << weights_precision));
    }    
    std::cout << data_i16[i] << std::endl;
}

-776
-6042
-1069
1970
-2066
-8430
3104
-5832
5227
5029
-6800
4405
1877
2981
1264
-8450
10831
7784
-8465
-1229


In [25]:
1.234 * (1 << 2)

4.9360000

In [31]:
float * tt_f32 = tt.data_ptr<float>();

for (int i=0; i<20; i++) {
    short v = data_i16[i];
    float w = tt_f32[i];
    auto f32 = float(v) / (1 << weights_precision);
    std::cout << f32 << " " << w << std::endl;
}

-0.0118408 -0.0118378
-0.0921936 -0.0921955
-0.0163116 -0.0163113
0.0300598 0.0300537
-0.0315247 -0.0315176
-0.128632 -0.128625
0.0473633 0.0473628
-0.0889893 -0.0889887
0.0797577 0.0797607
0.0767365 0.0767411
-0.10376 -0.103764
0.067215 0.0672154
0.0286407 0.0286361
0.0454865 0.0454809
0.0192871 0.0192807
-0.128937 -0.128944
0.165268 0.165265
0.118774 0.118772
-0.129166 -0.129173
-0.0187531 -0.0187604


In [32]:
short a = 0;
a

0

In [33]:
a = 1 << 15;
a

input_line_58:2:8: warning: implicit conversion from 'int' to 'short' changes value from 32768 to -32768 [-Wconstant-conversion]
 a = 1 << 15;
   ~ ~~^~~~~


-32768

In [34]:
unsigned short b = 1 << 15;

In [35]:
b

32768